In [1]:
%matplotlib inline
#! switch_R 3.3
%load_ext rpy2.ipython

In [2]:
import itertools
import math
from scipy import stats
import statsmodels.stats.api as sms
from pyspan.AFC.analysis import *
from pyspan import cross_validation as cv
from pyspan.ratings_task.analysis import freq_df, n_utterances, signal_df, SparseLR
from pyspan.utils import *
from pyspan.valence import *

/Users/sabinasloman/git/talking-politics/pyspan/config.py:19: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/traitlets/config/application.py", line 664, in launch_i

# Participants

In [3]:
n = len(minidf)
n_dem = len(minidf[minidf["party"] == "Democrat"])
n_repub = len(minidf[minidf["party"] == "Republican"])
print "N: {}".format(n)
print "N Dem: {}".format(n_dem)
print "N Repub: {}".format(n_repub)

N: 96
N Dem: 50
N Repub: 19


In [4]:
# Mean age and standard error of the mean
print np.mean(minidf.age[~np.isnan(minidf.age)]), stats.sem(minidf.age[~np.isnan(minidf.age)])
# n identifying as male/female
print len(minidf.loc[minidf.gender == 0]), len(minidf.loc[minidf.gender == 1])
# % voted
print np.mean(minidf.voted)

40.677083333333336 1.1210207198410143
37 59
0.8020833333333334


# Results

In [5]:
response_cols = map(str, p_ixs)
dat = minidf[response_cols]
responses = dat.values
dat[np.in1d(responses, truth["REPUBLICAN"]).reshape(responses.shape)] = 1
dat[dat != 1] = 0
dat["Condition"] = [ 1 if cond == "REPUBLICAN" else 0 for cond in 
                     minidf.Condition ]
dat_ = [ list(vals) for vals in dat.values ]

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/pandas/core/frame.py:3414: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [6]:
%%R -i dat_
library("lme4")
library("tidyverse")

d <- data.frame(t(dat_))
d.tbl <- d %>%
    mutate_each(as.integer) %>%
    mutate(condition=X88) %>%
    select(-X88) %>%
    mutate(pid=1:nrow(.)) %>%
    pivot_longer(paste0("X", 1:87), names_to="item")

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: Matrix

  warnings.warn(x, RRuntimeWarning)
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

  warnings.warn(x, RRuntimeWarning)
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: ✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

  warnings.warn(x, RRuntimeWarning)
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: ── Conflicts ──────────────────────────────────────────

## Main inferential tests

In [7]:
%%R
source("../clmclx.R")

In [8]:
%%R
# One-sided t-test of overall accuracy
d.tmp <- data.frame(d.tbl)
d.tmp[which(d.tmp$condition == 0),c("value")] = 1 - d.tmp[which(d.tmp$condition == 0),c("value")]
idxn <- with(d.tmp, pid[which(!is.na(value))])
idxk <- with(d.tmp, item[which(!is.na(value))])
d.tmp$value <- d.tmp$value - .5
fit <- lm(value ~ 1, data=d.tmp)
print(fit$df.residual)
res <- mclx(fit, 1, idxn, idxk)
mu <- res[1,1] + .5
se <- res[1,2]
t <- res[1,3]
p <- res[1,4] / 2
print(paste(mu, se, t, p))

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: zoo

  warnings.warn(x, RRuntimeWarning)
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching package: ‘zoo’


  warnings.warn(x, RRuntimeWarning)
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


  warnings.warn(x, RRuntimeWarning)


[1] 8351
[1] "0.520354406130268 0.0193591855470085 1.05140818454594 0.146550790601248"


In [9]:
%%R
# One-sided t-test (Republican condition)
d.tmp <- d.tbl %>%
    filter(condition == 1)
idxn <- with(d.tmp, pid[which(!is.na(value))])
idxk <- with(d.tmp, item[which(!is.na(value))])
d.tmp$value <- d.tmp$value - .5
fit <- lm(value ~ 1, data=d.tmp)
print(fit$df.residual)
res <- mclx(fit, 1, idxn, idxk)
mu <- res[1,1] + .5
se <- res[1,2]
t <- res[1,3]
p <- res[1,4] / 2
print(paste(mu, se, t, p))

[1] 4262
[1] "0.504339666901243 0.0244361100195502 0.177592378564808 0.429525774943945"


In [10]:
%%R
# One-sided t-test (Democratic condition)
# Responses of Republican are coded as 0, so the accuracy reported in the paper is
# 1-mu
d.tmp <- d.tbl %>%
    filter(condition == 0)
idxn <- with(d.tmp, pid[which(!is.na(value))])
idxk <- with(d.tmp, item[which(!is.na(value))])
d.tmp$value <- d.tmp$value - .5
fit <- lm(value ~ 1, data=d.tmp)
print(fit$df.residual)
res <- mclx(fit, 1, idxn, idxk)
mu <- res[1,1] + .5
se <- res[1,2]
t <- res[1,3]
p <- res[1,4] / 2
print(paste(mu, se, t, p))

[1] 4088
[1] "0.462949376375642 0.0203892640346995 -1.81716336407745 0.0346326434172826"


## Effect size

In [11]:
%%R
mod <- lmer(value ~ condition + (1|pid) + (condition|item), data=d.tbl)
delta <- summary(mod)$coefficients[2,1]
sig <- sigma(mod)
delta / sig

[1] 0.09018948


## Item-level analysis

In [12]:
perceps = PerceptualData()
perceps.get_discriminability_by_pair(plot = False)
mu, se = np.mean(perceps.discrim_by_pair.values()), \
    stats.sem(perceps.discrim_by_pair.values())
t, p = stats.ttest_1samp(perceps.discrim_by_pair.values(), .5)
mu, se, t, p / 2, len(perceps.discrim_by_pair.values()) - 1

(0.5202876083887881,
 0.01893722679498896,
 1.0713083076217071,
 0.14351406488368018,
 86)

Save data for figure (to reproduce figure, run `figures/figure3.py`).

In [13]:
item_dat = [ (k1,k2,v) for (k1,k2),v in perceps.discrim_by_pair.iteritems() ]
plot_df = pd.DataFrame(item_dat, columns=["dw","rw","acc"])
plot_df["logodds_d"] = [ signal_df.loc[w].rmetric for w in plot_df.dw ]
plot_df["logodds_r"] = [ signal_df.loc[w].rmetric for w in plot_df.rw ]
plot_df.to_csv("figure.csv")

Restrict to participants in the Republican condition.

In [14]:
perceps = PerceptualData(Condition = "REPUBLICAN")
perceps.get_discriminability_by_pair(plot = False)
mu, se = np.mean(perceps.discrim_by_pair.values()), \
    stats.sem(perceps.discrim_by_pair.values())
t, p = stats.ttest_1samp(perceps.discrim_by_pair.values(), .5)
mu, se, t, p / 2

(0.5044374071467668,
 0.023664392741587003,
 0.18751409322955584,
 0.4258496396573529)

Restrict to participants in the Democratic condition.

In [15]:
perceps = PerceptualData(Condition = "DEMOCRAT")
perceps.get_discriminability_by_pair(plot = False)
mu, se = np.mean(perceps.discrim_by_pair.values()), \
    stats.sem(perceps.discrim_by_pair.values())
t, p = stats.ttest_1samp(perceps.discrim_by_pair.values(), .5)
mu, se, t, p / 2

(0.5367369506735993,
 0.01998743469636781,
 1.8380022865203034,
 0.03475737716140077)

## Participant-level analysis

In [16]:
def get_idv_perceptions(i, ix, _df):
    response = _df.loc[i][str(ix)]
    if isinstance(response, str):
        assert response in partisan.values[ix-1]
        return int(response in truth[_df.loc[i]["Condition"]])
    else:
        return np.nan
    
def calc_prop_correct(_df):
    prop_correct = []
    for i in _df.index:
        z_idv_perceps = map(lambda ix: get_idv_perceptions(i, ix, _df), p_ixs)
        z_idv_perceps = np.array(z_idv_perceps)
        z_idv_perceps = z_idv_perceps[~np.isnan(z_idv_perceps)]
        prop_correct.append(sum(z_idv_perceps) / float(len(z_idv_perceps)))
    return prop_correct

In [17]:
prop_correct = calc_prop_correct(minidf)
mu, se = np.mean(prop_correct), stats.sem(prop_correct)
t, p = stats.ttest_1samp(prop_correct, .5)
mu, se, t, p / 2, len(prop_correct) - 1

(0.5203126392230242,
 0.006779377332631744,
 2.996239658360902,
 0.0017433022323757033,
 95)

Restrict to participants in the Republican condition.

In [18]:
prop_correct = calc_prop_correct(minidf.loc[minidf.Condition == "REPUBLICAN"])
mu, se = np.mean(prop_correct), stats.sem(prop_correct)
t, p = stats.ttest_1samp(prop_correct, .5)
mu, se, t, p / 2, len(prop_correct) - 1

(0.5044624104653891,
 0.009779850111900922,
 0.456286181723671,
 0.32512106460907353,
 48)

Restrict to participants in the Democrat condition.

In [19]:
prop_correct = calc_prop_correct(minidf.loc[minidf.Condition == "DEMOCRAT"])
mu, se = np.mean(prop_correct), stats.sem(prop_correct)
t, p = stats.ttest_1samp(prop_correct, .5)
mu, se, t, p / 2, len(prop_correct) - 1

(0.5368373458001331,
 0.00884265559541169,
 4.16586911054722,
 6.75282803752797e-05,
 46)

# Discussion

Performance on the Democratic words, broken down by the participant's party affiliation.

In [20]:
dat["party"] = minidf.party

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
prop_correct = calc_prop_correct(minidf.loc[(minidf.Condition == "DEMOCRAT") \
                                           & (minidf.party == "Democrat")])
mu, se = np.mean(prop_correct), stats.sem(prop_correct)
mu, se

(0.5225095785440613, 0.011862725568861674)

In [22]:
prop_correct = calc_prop_correct(minidf.loc[(minidf.Condition == "DEMOCRAT") \
                                           & (minidf.party == "Republican")])
mu, se = np.mean(prop_correct), stats.sem(prop_correct)
mu, se

(0.5426022453889335, 0.014358937218461763)

Mean $logodds_R$ of Democratic and Republican items.

In [23]:
np.mean(signal_df.loc[partisan.values[:,0]].rmetric), \
np.mean(signal_df.loc[partisan.values[:,1]].rmetric)

(-0.5806704207408324, 0.6424168695741973)

# Appendix E

Is there a significant difference in the accuracy of Democrats and Republicans?

In [24]:
accuracies = dict()
for party in ("Democrat", "Republican"):
    pdf = minidf[minidf["party"]==party]

    prop_correct = []
    for i in pdf.index:
        z_idv_perceps = map(lambda ix: get_idv_perceptions(i, ix, pdf), p_ixs)
        z_idv_perceps = np.array(z_idv_perceps)
        z_idv_perceps = z_idv_perceps[~np.isnan(z_idv_perceps)]
        prop_correct.append(sum(z_idv_perceps) / float(len(z_idv_perceps)))
    accuracies[party] = prop_correct

In [25]:
d, r = accuracies["Democrat"], accuracies["Republican"]

print "Mean discriminability for Democrats: %f (SE: %f)"%(np.mean(d), stats.sem(d))
print "Mean discriminability for Republicans: %f (SE: %f)"%(np.mean(r), stats.sem(r))
print "\n"

cm = sms.CompareMeans(sms.DescrStatsW(d), sms.DescrStatsW(r))
tstat, pvalue, deg_freedom = cm.ttest_ind(usevar="unequal")
print "T-stat: %f"%tstat
print "P value: %f"%pvalue
print "Degrees of freedom: %f"%deg_freedom

Mean discriminability for Democrats: 0.508276 (SE: 0.009302)
Mean discriminability for Republicans: 0.534469 (SE: 0.015246)


T-stat: -1.466555
P value: 0.152180
Degrees of freedom: 32.252885


In [26]:
signal_df = pickle.load(open(paths["metrics_dir"] + "signals-unigrams", "rb"))

def get_signal(w):
    dlogp, rlogp = signal_df.loc[w]["dmetric"], signal_df.loc[w]["rmetric"]
    assert dlogp * rlogp < 0
    return dlogp, rlogp

signals_ = map(get_signal, itertools.chain(*partisan_pairs))
signals = dict(zip(itertools.chain(*partisan_pairs), signals_))

In [27]:
lr_data = minidf.copy()
lr_data = lr_data.loc[lr_data.party.isin([ "Democrat", "Republican" ])]
n = len(lr_data)
partisan_pairs = np.array(partisan_pairs)
n_vars = 9

# Add dummy columns
for i in range(1, n):
    ids = np.zeros(n)
    ids[i] = 1
    lr_data["participant{}".format(i)] = ids
    
w1 = np.ravel(lr_data[map(str, p_ixs)])
w2 = w1.copy()
for pair in partisan_pairs:
    w2[w2 == pair[0]] = "TMP"
    w2[w2 == pair[1]] = pair[0]
    w2[w2 == "TMP"] = pair[1]
    
X = np.full((n * len(p_ixs), n_vars + n - 1 + len(p_ixs)-1), np.nan)
# The log odds that the word was said by a Republican
vf = np.vectorize(lambda w1_, w2_: signals[w1_][0] + signals[w2_][1])
X[:,0] = np.tile(vf(partisan_pairs[:,0], partisan_pairs[:,1]), n)
# The log probability of hearing the word
vf = np.vectorize(lambda w1_, w2_: math.log(sum(freq_df.loc[(w1_,["dmetric","rmetric"])].values) / n_utterances, 2) +
                                   math.log(sum(freq_df.loc[(w2_,["dmetric","rmetric"])].values) / n_utterances, 2))
X[:,1] = np.tile(vf(partisan_pairs[:,0], partisan_pairs[:,1]), n)
# Participant's political identity
vf = np.vectorize(lambda pid: 1 if pid == "Republican" else -1 if pid == "Democrat" else 
                  0)
pids = np.repeat(vf(lr_data.party), len(p_ixs))
X[:,2] = pids
# Valence of word
vf = np.vectorize(lambda w1_, w2_: get_valence(w1_)[0]-5 - (get_valence(w2_)[0]-5))
X[:,3] = vf(*np.array(zip(w1, w2)).T)
# Valence of word * Participant's party identity * Polarity
sign = np.vectorize(lambda w: np.sign(signal_df.loc[w].rmetric) if not is_nan(w) else np.nan)
X[:,4] = pids * X[:,3] * sign(w1)
# Party identity
X[:,5] = np.repeat(lr_data.party_identity, len(p_ixs))
# Party affiliation x party identity
X[:,6] = pids * X[:,5]
# Political engagement
X[:,7] = np.repeat(lr_data.political_engagement, len(p_ixs))
# Party affiliation x political engagement
X[:,8] = pids * X[:,7]
# Participant IDs
for i in range(n_vars, n_vars + n - 1):
    X[:,i] = np.repeat(lr_data["participant{}".format(i-n_vars+1)], len(p_ixs))
    
# Item dummies
for i in range(1,len(p_ixs)):
    ix = n_vars+n-1+i-1
    ind_vec = np.zeros(len(p_ixs))
    ind_vec[i] = 1
    X[:,ix] = np.tile(ind_vec, n)

condition = np.repeat(lr_data.Condition, len(p_ixs))
condition[condition == "DEMOCRAT"] = -1
condition[condition == "REPUBLICAN"] = 1
Y = sign(w1) == condition

X = X[~np.isnan(Y)]
Y = Y[~np.isnan(Y)]
Y = Y[~np.isnan(X).any(axis = 1)]
X = X[~np.isnan(X).any(axis = 1),:]

# Since we don't have valence data for some of the words, their observations will be deleted and their indicator columns will
# just be clutter (and cause problems with model estimation)
colixs = np.arange(0,X.shape[1])
to_del = np.where(np.apply_along_axis(lambda x: np.all(x == 0), 0, X[:,n_vars+n-1:-1]))[0] + n_vars+n-1
colixs = [ colix for colix in colixs if colix not in to_del ]
X = X[:,colixs]

X = stats.mstats.zscore(X)

In [28]:
# Estimate the model without fixed effects for consistency with the analysis of the Study 1 data
logit = SparseLR(Y, X[:,:n_vars])
logit.coef, logit.n, logit.auc

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


(array([ 0.17068798,  0.06898615,  0.03999893, -0.02842231,  0.33605812,
        -0.02959563, -0.03028575,  0.07660507,  0.01620017]),
 5381,
 0.5983633537608953)